In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# How to use PyTorch LSTMs for time series regression

# Data

1. Download the data from the URLs listed in the docstring in `preprocess_data.py`.
2. Run the `preprocess_data.py` script to compile the individual sensors PM2.5 data into
   a single DataFrame.

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/SoftExo/Repositories/trajectory_prediction_EMG/dataset/allemgdata.csv')
# df = pd.read_csv('dataset/allemgdata.csv')
df['pos_elbow'] = df['pos_elbow'] - df['pos_elbow'].min()
df['pos_shfe'] = df['pos_shfe'] - df['pos_shfe'].min()
df['pos_shaa'] = df['pos_shaa'] - df['pos_shaa'].min()
df

,emg_elbow,emg_shfe,emg_shaa,pos_elbow,pos_shfe,pos_shaa
0,13.698,11.835,8.3784,0.061359,0.049088,0.595180
1,13.706,11.901,8.3871,0.061359,0.049088,0.589050
2,13.716,11.970,8.3912,0.061359,0.049088,0.582910
3,13.742,12.055,8.3937,0.061359,0.049088,0.576780
4,13.762,12.146,8.4052,0.061359,0.049088,0.570640
...,...,...,...,...,...,...
65939,154.180,127.530,145.7300,0.092039,0.128852,0.067496
65940,152.720,126.040,144.1700,0.092039,0.128852,0.067496
65941,151.510,124.770,142.8900,0.092039,0.128852,0.067496
65942,150.350,123.420,141.7400,0.092039,0.128852,0.067496


In [3]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(df[['emg_elbow','emg_shfe','emg_shaa']][0:10000], labels=dict(
    created_at="Date", value="Amplitude", variable="Sensor"
))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()
# fig.write_image("emgdata.png", width=1200, height=600)

In [4]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(df[['pos_elbow','pos_shfe','pos_shaa']][0:10000], labels=dict(
    created_at="Date", value="Amplitude", variable="Sensor"
))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

## Create the target variable

Here we are shifting the data n times ahead

In [5]:
target_sensor = ['pos_elbow','pos_shfe','pos_shaa']

# If you only want EMG data as features:
# features = list(df.columns.difference([target_sensor]))

# If you want both position and EMG as features:
features = list(df.columns[0:6])
print("our features are: ", features)
    
forecast_lead = 15
target = f"pos_elbow_lead{forecast_lead} pos_shfe_lead{forecast_lead} pos_shaa_lead{forecast_lead}"
target = target.split()
print("our target to predict is: ", target)

df[target] = df[target_sensor].shift(-forecast_lead)
df = df.iloc[:-forecast_lead]


our features are:  ['emg_elbow', 'emg_shfe', 'emg_shaa', 'pos_elbow', 'pos_shfe', 'pos_shaa']
our target to predict is:  ['pos_elbow_lead15', 'pos_shfe_lead15', 'pos_shaa_lead15']


## Create a hold-out test set and preprocess the data

In [6]:
test_start = 55000

df_train = df.loc[:test_start].copy()
df_test = df.loc[test_start:].copy()

print("Test set fraction:", len(df_test) / len(df))

Test set fraction: 0.16576923660301232


## Standardize the features and target, based on the training set

In [7]:
target_mean = df_train[target].mean()
target_stdev = df_train[target].std()

for c in df_train.columns:
    mean = df_train[c].mean()
    stdev = df_train[c].std()

    df_train[c] = (df_train[c] - mean) / stdev
    df_test[c] = (df_test[c] - mean) / stdev

## Create datasets that PyTorch `DataLoader` can work with

In [8]:
import torch
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[self.target].values).float()
        self.X = torch.tensor(dataframe[self.features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [9]:
i = 100
sequence_length = 5

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)

X, y = train_dataset[i]
print(X)
print(y)

tensor([[ 0.6801,  0.8671,  0.2083, -0.6180, -0.5575, -0.4914],
        [ 0.7055,  0.9065,  0.2464, -0.6180, -0.5575, -0.4914],
        [ 0.7388,  0.9529,  0.2902, -0.6180, -0.5575, -0.5051],
        [ 0.7641,  0.9898,  0.3271, -0.6180, -0.5575, -0.5051],
        [ 0.7801,  1.0161,  0.3565, -0.6180, -0.5575, -0.5051]])
tensor([-0.6181, -0.5575, -0.5458])


In [10]:
X, y = train_dataset[i + 1]
print(X)

tensor([[ 0.7055,  0.9065,  0.2464, -0.6180, -0.5575, -0.4914],
        [ 0.7388,  0.9529,  0.2902, -0.6180, -0.5575, -0.5051],
        [ 0.7641,  0.9898,  0.3271, -0.6180, -0.5575, -0.5051],
        [ 0.7801,  1.0161,  0.3565, -0.6180, -0.5575, -0.5051],
        [ 0.7978,  1.0433,  0.3814, -0.6180, -0.5575, -0.5051]])


In [11]:
print(df_train[features].iloc[(i - sequence_length + 1): (i + 1)])

     emg_elbow  emg_shfe  emg_shaa  pos_elbow  pos_shfe  pos_shaa
96    0.680081  0.867111  0.208349  -0.618048 -0.557476 -0.491441
97    0.705479  0.906508  0.246409  -0.618048 -0.557476 -0.491441
98    0.738778  0.952904  0.290186  -0.618048 -0.557476 -0.505121
99    0.764058  0.989843  0.327100  -0.618048 -0.557476 -0.505121
100   0.780096  1.016128  0.356539  -0.618048 -0.557476 -0.505121


In [12]:
from torch.utils.data import DataLoader
torch.manual_seed(99)

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)

X, y = next(iter(train_loader))
print(X.shape)
print(X)

torch.Size([3, 5, 6])
tensor([[[ 0.0199,  0.0167,  0.0154, -0.4867, -0.6088, -0.7103],
         [-0.0148, -0.0130, -0.0141, -0.4867, -0.6088, -0.7103],
         [-0.0489, -0.0419, -0.0431, -0.4867, -0.6088, -0.7103],
         [-0.0822, -0.0699, -0.0713, -0.4867, -0.6088, -0.7103],
         [-0.1149, -0.0974, -0.0986, -0.4867, -0.6088, -0.7103]],

        [[ 0.9909,  0.5562,  1.1272, -0.5805, -0.0186, -0.7240],
         [ 0.9898,  0.5700,  1.1261, -0.5805, -0.0186, -0.7240],
         [ 0.9982,  0.5907,  1.1351, -0.5805, -0.0186, -0.7240],
         [ 1.0012,  0.6050,  1.1383, -0.5805, -0.0186, -0.7240],
         [ 0.9971,  0.6150,  1.1343, -0.5805, -0.0186, -0.7240]],

        [[-0.8373, -0.6315, -0.7880,  1.4840,  1.3671,  2.0803],
         [-0.8387, -0.6291, -0.7870,  1.4652,  1.3671,  2.0803],
         [-0.8406, -0.6264, -0.7863,  1.4652,  1.3415,  2.0803],
         [-0.8425, -0.6245, -0.7854,  1.4464,  1.3415,  2.0803],
         [-0.8452, -0.6234, -0.7847,  1.4464,  1.3415,  2.0803]]

## Create the datasets and data loaders for real

Using just 4 time periods to forecast 15 time periods ahead seems challenging, so let's
use sequences of length 30 (60 minutes) instead.

The PyTorch `DataLoader` is a very convenient way to iterate through these datasets. For
the training set we'll shuffle (the rows *within* each training sequence are not
shuffled, only the order in which we draw those blocks). For the test set, shuffling
isn't necessary.

In [13]:
torch.manual_seed(101)

batch_size = 25
sequence_length = 30

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(test_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([25, 30, 6])
Target shape: torch.Size([25, 3])


# The model and learning algorithm

Most importantly, we have to keep track of which dimension represents the batch in our input tensors. As we just saw, our data loaders use the first dimension for this, but the PyTorch LSTM layer’s default is to use the second dimension instead. So we set batch_first=True to make the dimensions line up, but confusingly, this doesn’t apply to the hidden and cell state tensors. In the forward method, we initialize h0 and c0 with batch size as the second dimension.

In [14]:
from torch import nn 

class CNNLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )
        
        # input "image" size here is torch.Size([25, 1, 30, 6])
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (5, 3), stride=(2, 1), padding=(4, 2))
        # height: input_size-filter_size +2(padding)/stride + 1 = 30-5+2(4)/2+1=17
        # width: 6-3+2*2/1 + 1 = 8
        # torch.Size([25, 8, 17, 8])
        self.batchnorm1 = nn.BatchNorm2d(8)
        # torch.Size([25, 8, 17, 8])
        # xcnnput_channel:8, batch(8)
        self.relu = nn.ReLU()
        # torch.Size([25, 8, 17, 8])
        self.maxpool1 = nn.MaxPool2d(kernel_size=(4,2))
        # torch.Size([25, 8, 4, 4])
        #input_size=28/2=14
        self.cnn2 = nn.Conv2d(in_channels=8, out_channels=32, kernel_size = (5, 1), stride=(2, 1), padding=(1, 1))
        # same_padding: (5-1)/2=2:padding_size. 
        # torch.Size([25, 32, 1, 6])
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.maxpool2 = nn.MaxPool2d(kernel_size=1)
        self.fc1 = nn.Linear(in_features=192, out_features=96)
        # Nx3 * 3xO = NxO
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 =nn.Linear(in_features=96, out_features=3)
        self.fc3 =nn.Linear(in_features=num_sensors, out_features=3)

        self.linear = nn.Linear(
            in_features=self.hidden_units, 
            out_features=3
            )

    def forward(self, x):
        
        batch_size = x.shape[0]
        
        # The CNN part:
        xcnn = x
        xcnn = xcnn.reshape(batch_size,1,x.shape[1],x.shape[2])
        xcnn = self.cnn1(xcnn)
        xcnn =self.batchnorm1(xcnn)
        xcnn =self.relu(xcnn)
        xcnn =self.maxpool1(xcnn)
        xcnn =self.cnn2(xcnn)
        xcnn =self.batchnorm2(xcnn)
        xcnn =self.relu(xcnn)
        xcnn =self.maxpool2(xcnn)
        xcnn = torch.flatten(xcnn,1)
        xcnn =self.fc1(xcnn)
        xcnn =self.relu(xcnn)
        xcnn =self.dropout(xcnn)
        xcnn =self.fc2(xcnn)
        
        # The LSTM part
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()  # torch.Size([1, 25, 16])
        
        _, (hn, _) = self.lstm(x, (h0, c0))
        xlstm = self.linear(hn[0])  # First dim of Hn is num_layers, which is set to 3 above.

        
        # concat
        out = torch.cat((xcnn,xlstm), 1)
        out  = self.fc3(out)
        
        return out


In [15]:
learning_rate = 5e-5
num_hidden_units = 16

model = CNNLSTM(num_sensors=len(features), hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
def saveModel():
    path = '/content/drive/MyDrive/SoftExo/Repositories/trajectory_prediction_EMG/models/CNNLSTM.pth'
    torch.save(model.state_dict(), path)

# Train

In [17]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    saveModel()
    print(f"Train loss: {avg_loss}")

def test_model(data_loader, model, loss_function):
    
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            # For my information, X.shape = torch.Size([25, 30, 6]) and y.shape = ([25, 3])
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")

In [18]:
print("Untrained test\n--------")
test_model(test_loader, model, loss_function)
print()

epoch_number = 50

for ix_epoch in range(epoch_number):
    print(f"Epoch {ix_epoch}\n---------")
    train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_model(test_loader, model, loss_function)
    print()

Untrained test
--------
Test loss: 1.0226189188455066

Epoch 0
---------
Train loss: 0.34226860774038653
Test loss: 0.10174647082051594

Epoch 1
---------
Train loss: 0.08683711703086457
Test loss: 0.05158743151231269

Epoch 2
---------
Train loss: 0.05381061987480024
Test loss: 0.04294346621070075

Epoch 3
---------
Train loss: 0.04202792259260694
Test loss: 0.030426043213365575

Epoch 4
---------
Train loss: 0.033805495124650455
Test loss: 0.02417701826942234

Epoch 5
---------
Train loss: 0.027729976885249228
Test loss: 0.0215006764765141

Epoch 6
---------
Train loss: 0.022304479233109846
Test loss: 0.0174799584624639

Epoch 7
---------
Train loss: 0.018615645529962112
Test loss: 0.018083954069111542

Epoch 8
---------
Train loss: 0.015808820630551088
Test loss: 0.017151741501524383

Epoch 9
---------
Train loss: 0.013275211920787405
Test loss: 0.016225161029061262

Epoch 10
---------
Train loss: 0.011670458696588793
Test loss: 0.01594022687738982

Epoch 11
---------
Train loss: 0.

# Evaluation

In [19]:
def predict(data_loader, model):
    """Just like `test_loop` function but keep track of the outputs instead of the loss
    function.
    """
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output

In [20]:
train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

ystar_col = ['predpos_elbow', 'predpos_shfe', 'predpos_shaa']
df_train[ystar_col] = predict(train_eval_loader, model).numpy()
df_test[ystar_col] = predict(test_loader, model).numpy()

In [21]:
target = ['pos_elbow_lead15', 'pos_shfe_lead15', 'pos_shaa_lead15']
df_out_elbow = pd.concat((df_train, df_test))[['pos_elbow_lead15', 'predpos_elbow']]

In [22]:
# df_out = pd.concat(df_train[target], df_train[ystar_col])

for c in df_out_elbow.columns:
    df_out_elbow[c] = df_out_elbow[c] * target_stdev[0] + target_mean[0]

print(df_out_elbow)

       pos_elbow_lead15  predpos_elbow
0              0.061359       0.064918
1              0.061359       0.065033
2              0.061359       0.065363
3              0.061359       0.065669
4              0.061359       0.066333
...                 ...            ...
65924          0.092039       0.105593
65925          0.092039       0.104970
65926          0.092039       0.104934
65927          0.092039       0.104150
65928          0.092039       0.103695

[65930 rows x 2 columns]


In [23]:
xaxis = df_out_elbow.index * 0.002 
fig = px.line(df_out_elbow, labels={'value': "angular position of elbow (rad)", 'created_at': 'Date'})
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
fig.add_annotation(xref="paper", x=0.75, yref="paper", y=0.8, text="Test set start", showarrow=False)
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()
# fig.write_image("pm25_forecast.png", width=1200, height=600)